In [ ]:
#pip install stanza

In [1]:
import os
import json
import stanza
import csv

# Build an English pipeline
stanza.download('en') # download English model
nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,ner', package={"ner": ["ncbi_disease", "ontonotes"]}) # initialize English neural pipeline

C:\Users\okechukwu chude\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-12 14:48:01 INFO: Downloaded file to C:\Users\okechukwu chude\stanza_resources\resources.json
2024-03-12 14:48:01 INFO: Downloading default packages for language: en (English) ...
2024-03-12 14:48:03 INFO: File exists: C:\Users\okechukwu chude\stanza_resources\en\default.zip
2024-03-12 14:48:06 INFO: Finished downloading models and saved to C:\Users\okechukwu chude\stanza_resources
2024-03-12 14:48:06 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-03-12 14:48:07 INFO: Downloaded fi

In [2]:
# Global variable to store all extracted texts
extracted_texts = []

In [3]:
def process_text_with_stanza(text):
    doc = nlp(text)
    return doc


In [4]:
def extract_info_from_json(json_file_path, hadm_id_set):
    global extracted_texts

    # Open the JSON file
    with open(json_file_path, 'r') as file:
        # Load JSON data from the file
        data = json.load(file)

        # Extract 'hadm_id' and 'comment' from the JSON data
        hadm_id = data.get('hadm_id', None)
        comment = data.get('comment', None)

        # If 'hadm_id' is not found, print a warning message
        if hadm_id is None:
            print(f"Warning: 'hadm_id' not found in {json_file_path}")
            return None

        # Add 'hadm_id' to the set
        hadm_id_set.add(hadm_id)

        # Extract 'notes' from the JSON data
        notes = data.get('notes', [])

        # Iterate through each note
        for note in notes:
            note_info = {}  # Initialize a dictionary to store note information

            # Add 'hadm_id' to the note information
            note_info['hadm_id'] = hadm_id

            # Extract information from the note
            note_info['note_id'] = note.get('note_id', None)
            note_info['category'] = note.get('category', None)
            note_info['description'] = note.get('description', None)

            # Extract annotations from the note
            annotations = note.get('annotations', [])
            annotations_info = []  # Initialize a list to store annotation information

            # Iterate through each annotation in the note
            for annotation in annotations:
                annotation_info = {}  # Initialize a dictionary to store annotation information

                # Extract information from the annotation
                annotation_info['begin'] = annotation.get('begin', None)
                annotation_info['end'] = annotation.get('end', None)
                annotation_info['code'] = annotation.get('code', None)
                annotation_info['code_system'] = annotation.get('code_system', None)
                annotation_info['description'] = annotation.get('description', None)
                annotation_info['type'] = annotation.get('type', None)
                annotation_info['covered_text'] = annotation.get('covered_text', None)

                annotations_info.append(annotation_info)  # Append annotation information to the list

            note_info['annotations'] = annotations_info  # Add annotations information to the note
            note_info['text'] = note.get('text', None)

            # Process text with Stanza
            if note_info['text']:
                processed_text = process_text_with_stanza(note_info['text'])
                note_info['processed_text'] = processed_text

                # Count number of sentences and store it
                num_sentences = len(processed_text.sentences)
                note_info['num_sentences'] = num_sentences

            extracted_texts.append(note_info)  # Append note information to the global variable

In [5]:
def search_files(folder_path):
    # Set to store unique hadm_id values
    hadm_id_set = set()

    # Recursively search for JSON files in the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.json'):
                # Construct the full path to the JSON file
                json_file_path = os.path.join(root, filename)
                print("Processing:", json_file_path)
                # Call extract_info_from_json function to extract information from the JSON file
                extract_info_from_json(json_file_path, hadm_id_set)
                # Print a separator after processing each file
                print("=" * 50)

    # Print the count of unique hadm_id values
    print("Total unique hadm_id count:", len(hadm_id_set))

    # # Write extracted information to CSV file
    # write_to_csv(extracted_texts)


In [ ]:
# Specify the path to the main folder containing subfolders with JSON files
main_folder_path = r"C:\Users\okechukwu chude\Documents\NLP\text extraction\Automating-Medical-Coding\testing"

# Call the search_files function to start searching for JSON files in the main folder and its subfolders
search_files(main_folder_path)

In [ ]:
# Calculate total number of sentences
total_sentences = sum(note_info['num_sentences'] for note_info in extracted_texts if 'num_sentences' in note_info)

# Print total number of sentences
print("Total number of sentences:", total_sentences)

In [ ]:
#pip install transformers

**Sentence Tokenisation**

In [ ]:
from stanza.utils.conll import CoNLL

def tokenize_sentences(doc):

    tokenized_sentences = []  # Initialize an empty list to store tokenized sentences
    for sentence in doc.sentences:  # Iterate through each sentence in the document
        tokens = [token.text for token in sentence.tokens]  # Extract text of each token in the sentence
        tokenized_sentences.append(tokens)  # Append the list of tokens to the tokenized_sentences list
    return tokenized_sentences  # Return the tokenized sentences

# Loop through each note_info in the extracted_texts list
for note_info in extracted_texts:
    # Check if the 'processed_text' key exists in the current note_info
    if 'processed_text' in note_info:
        processed_text = note_info['processed_text']  # Extract the processed text from note_info
        tokenized_sentences = tokenize_sentences(processed_text)  # Tokenize the sentences in the processed text
        note_info['tokenized_sentences'] = tokenized_sentences  # Add the tokenized sentences to note_info

# Loop through each note_info in the extracted_texts list
for i, note_info in enumerate(extracted_texts):
    if i >= 10:  # Stop after processing the first 50 notes
        break

    # Check if the 'tokenized_sentences' key exists in the current note_info
    if 'tokenized_sentences' in note_info:
        print(f"HADM_ID: {note_info['hadm_id']}")  # Print the HADM_ID
        print(f"Note ID: {note_info['note_id']}")  # Print the note ID
        tokenized_sentences = note_info['tokenized_sentences']  # Extract the tokenized sentences from note_info
        # Loop through each sentence_tokens list in the tokenized_sentences list
        for sentence_tokens in tokenized_sentences:
            print(' '.join(sentence_tokens))  # Print the tokens of each sentence separated by spaces
        print()  # Print an empty line after printing all sentences

In [ ]:
# Initialize total word count
total_word_count = 0

# Loop through each note_info in the extracted_texts list
for note_info in extracted_texts:
    # Check if the 'tokenized_sentences' key exists in the current note_info
    if 'tokenized_sentences' in note_info:
        tokenized_sentences = note_info['tokenized_sentences']  # Extract the tokenized sentences from note_info

        # Iterate through each sentence_tokens list in the tokenized_sentences list
        for sentence_tokens in tokenized_sentences:
            # Increment the total word count by the number of words in the current sentence
            total_word_count += len(sentence_tokens)

# Print the total number of words
print("Total number of words:", total_word_count)


**PART OF SPEECH TAGGING**

NER DATA FORMAT

In [ ]:
import pandas as pd


data = []

for note_info in extracted_texts:
    if 'tokenized_sentences' in note_info and 'annotations' in note_info:
        tokenized_sentences = note_info['tokenized_sentences']
        annotations = note_info['annotations']

        sentence_id = 0
        for sentence_tokens in tokenized_sentences:
            words = sentence_tokens
            labels = ['O'] * len(words)  # Initialize all labels as 'O' (Outside)

            # Match annotations with words in the sentence
            for annotation in annotations:
                begin = annotation['begin']
                end = annotation['end']
                code = annotation['code']
                covered_text = annotation['covered_text']

                # Find the word indices corresponding to the annotation
                word_indices = []
                for i, word in enumerate(words):
                    if covered_text.startswith(word):
                        word_indices.append(i)
                        covered_text = covered_text[len(word):]
                        if not covered_text:
                            break

                # Update labels for the matched words
                for idx in word_indices:
                    labels[idx] = f'B-{code}' if idx == word_indices[0] else f'I-{code}'

            # Append data for each word in the sentence
            for word, label in zip(words, labels):
                data.append({'sentence_id': sentence_id, 'words': word, 'labels': label})

            sentence_id += 1

# Create the DataFrame
ner_df = pd.DataFrame(data)

In [ ]:
ner_df.head()

In [ ]:
# Count the occurrences of each unique value in the 'labels' column
label_counts = ner_df['labels'].value_counts()

# Display the counts
print(label_counts)

In [ ]:
# Calculate the total number of evidence words
total_evidence_words = total_word_count - label_counts.get('O', 0)

# Print the total number of evidence words
print("Total number of evidence words:", total_evidence_words)


EXTRACT EVIDENCE FROM THE JSON FILES

In [ ]:
def search_json_files(folder_path):
    # Initialize a list to store all JSON file paths
    json_files = []

    # Recursively search for JSON files in the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))

    return json_files


In [ ]:
# # Initialize a global notes DataFrame
# global_df = pd.DataFrame()

In [ ]:
def generate_labels_from_json(json_file):
    # Open the JSON file
    with open(json_file, 'r') as file:
        # Load JSON data from the file
        data = json.load(file)

        # Extract 'notes' from the JSON data
        notes = data.get('notes', [])

        # Iterate through each note
        for note in notes:
            note_text = note.get('text', None)
            if note_text:
                # Tokenize the note text
                tokens = tokenize_text(note_text)

                # Extract annotations from the note
                annotations = note.get('annotations', [])

                # Initialize labels for tokens
                labels = ['O'] * len(tokens)

                # Match annotations with tokens
                for annotation in annotations:
                    begin = annotation['begin']
                    end = annotation['end']
                    code = annotation['code']
                    covered_text = annotation['covered_text']

                    # Find the token indices corresponding to the annotation
                    token_indices = find_token_indices(tokens, begin, end, covered_text)

                    # Update labels for the matched tokens
                    for idx in token_indices:
                        labels[idx] = f'B-{code}' if idx == token_indices[0] else f'I-{code}'



In [ ]:
def tokenize_text(text):
    # Process the text with the initialized pipeline
    doc = nlp(text)
    # Extract tokens from the document
    tokens = [word.text for sent in doc.sentences for word in sent.words]
    print(doc.ents)

    return tokens

Tokenization and Sentence Segmentation

In [ ]:
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')

In [ ]:
# Spath to the main folder containing subfolders with JSON files
main_folder_path = r"C:\Users\okechukwu chude\Documents\NLP\text extraction\Automating-Medical-Coding\testing"

# Search for JSON files in the main folder and its subfolders
json_files = search_json_files(main_folder_path)

# Iterate through each JSON file and generate labels for tokens
for json_file in json_files:
    print(f"Processing {json_file}...")
    generate_labels_from_json(json_file)
